In [1]:
import camelot
import pandas as pd
import numpy as np
import re
import os

In [2]:
# Helper codes

def clean_country_name(name):
    name = str(name).strip()              
    name = re.sub(r'\s+', ' ', name)     
    name = re.sub(r"\s*-\s*.*", "", name)
    name = re.sub(r"[^A-Za-z\s]", "", name)
    name = re.sub(r'[^A-Za-z\s]', '', name)  
    return name.title()                   

# --- Constants ---
MONTH_ORDER = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

# --- Helpers ---
def get_months(df):
    """Return ordered list of existing month columns."""
    return [m for m in MONTH_ORDER if m in df.columns]


In [3]:
exclude_words = [
    "A S I A",
    "ASEAN",
    "SUB-TOTAL",
    "EAST ASIA",
    "SOUTH ASIA",
    "MIDDLE EAST",
    "A M E R I C A",
    "NORTH AMERICA",
    "SOUTH AMERICA",
    "E U R O P E",
    "WESTERN EUROPE",
    "NORTHERN EUROPE",
    "SOUTHERN EUROPE",
    "EASTERN EUROPE",
    "EASTERN MEDITERRANEAN EUROPE",
    "AUSTRALASIA/PACIFIC",
    "TOTAL (CIS & RUSSIA)"
    "A F R I C A	"
]

In [4]:
# Read tables from PDF (PAGE 1)
pdf_file = "/Users/kim/Desktop/repos/Philippines_Visitor/data/rawData/2018-JAN-DEC.pdf"

tables = camelot.read_pdf(pdf_file, pages='1', flavor="stream")

# Combine all extracted tables into one DataFrame
df1 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df1 = df1.iloc[2:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df1.columns = range(df1.shape[1])

df1.drop(
    df1[df1[0].str.upper().isin(exclude_words)].index,
    inplace=True
)

df1 = df1.reset_index(drop=True).iloc[:-1]


In [5]:
# Read tables from PDF (PAGE 2)
tables = camelot.read_pdf(pdf_file, pages='2', flavor="stream")

# Combine all extracted tables into one DataFrame
df2 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df2 = df2.iloc[3:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df2.columns = range(df2.shape[1])

df2.drop(
   df2[df2[0].str.upper().isin(exclude_words)].index,
    inplace=True
)

df2 = df2[df2[0] != 'EASTERN MEDITERRANEAN EUROPE**'].reset_index(drop=True)

# DROP TOTAL RUSSIAN FEDERATION 
df2 = df2.reset_index(drop=True)
df2 = df2.drop(26).reset_index(drop=True)
df2 = df2.drop(23).reset_index(drop=True)
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,ARGENTINA,586,513,457,422,294,177,173,145,150,161,182,208,"3,468",0.05,"3,157",9.85
1,BRAZIL,776,709,701,723,539,420,456,469,476,477,601,"1,017","7,364",0.10,"6,114",20.44
2,COLOMBIA,152,154,167,185,187,161,221,238,228,242,224,230,"2,389",0.03,"1,747",36.75
3,PERU,82,82,104,84,59,58,69,107,87,112,87,107,"1,038",0.01,"1,008",2.98
4,VENEZUELA,37,18,36,33,24,25,35,26,34,61,50,44,423,0.01,257,64.59
5,ANDORRA,38,25,35,30,44,50,47,2,-,2,,-,273,0.00,357,-23.53
6,AUSTRIA,"1,742","1,435","1,204","1,151",924,750,"1,446",899,761,891,"1,069","1,921","14,193",0.20,"13,524",4.95
7,BELGIUM,"1,493","1,353","1,575","1,554","1,223","1,474","2,241","1,098",953,"1,181","1,393","1,747","17,285",0.24,"15,703",10.07
8,FRANCE,"8,438","9,737","8,489","8,628","5,561","3,339","5,253","4,556","3,397","4,580","5,641","6,781","74,400",1.04,"64,777",14.86
9,GERMANY,"8,822","9,826","10,161","7,551","6,532","4,828","6,413","6,127","5,484","6,733","7,888","11,733","92,098",1.28,"85,431",7.80


In [6]:
# Read tables from PDF (PAGE 3)
tables = camelot.read_pdf(pdf_file, pages='3', flavor="stream")

# Combine all extracted tables into one DataFrame
df3 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df3 = df3.iloc[3:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df3.columns = range(df3.shape[1])

df3.drop(
   df3[df3[0].str.upper().isin(exclude_words)].index,
    inplace=True
)
df3 = df3.drop([6,10]).reset_index(drop=True)
df3 = df3.reset_index(drop=True)
df3 = df3.iloc[:-20]


In [7]:
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

combined_df.columns = [
        "Country",
        "January",
        "February",
        "March",
        "April",
        "May",
        "June",
        "July",
        "August",
        "September",
        "October",
        "November",
        "December",
        "Total",
        "Percentage",
        "Previous Total",
        "Growth Rate"
        ]

# Clean and convert column to float

combined_df['Total'] = (
    combined_df['Total']
    .astype(str)               # ensure everything is string
    .str.replace(',', '')      # remove commas
    .str.strip()               # remove leading/trailing spaces
    .replace({'': np.nan, '-': np.nan})  # replace empty strings and '-' with NaN
    .astype(float)             # convert to float
)
combined_df = combined_df.sort_values(by="Total", ascending=False).reset_index(drop=True)

# Copy the row you want to move
row_to_move = combined_df.iloc[16]

# Drop the original row
combined_df = combined_df.drop(16).reset_index(drop=True)

top = combined_df.iloc[:1]   
bottom = combined_df.iloc[1:] 

# Insert the row
combined_df = pd.concat([top, pd.DataFrame([row_to_move]), bottom]).reset_index(drop=True)

combined_df['Country'] = combined_df['Country'].apply(clean_country_name)
combined_df['Country'] = combined_df['Country'].replace('Independent States', 'Commonwealth of Independent States')
combined_df['Country'] = combined_df['Country'].replace('Hongkong', 'Hong Kong')
combined_df['Country'] = combined_df['Country'].replace('Korea', 'South Korea')

combined_df.to_csv("/Users/kim/Desktop/repos/Philippines_Visitor/data/cleanedData/2018cleanData.csv")


In [8]:
# Store Monthly Data
monthly_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/monthly_visitors"

monthly_df = combined_df.iloc[2:].reset_index(drop=True) 
monthly_df['Country'] = monthly_df['Country'].apply(clean_country_name)

months = get_months(combined_df)

monthly_df = monthly_df[['Country'] + months]

mon_out_path = os.path.join(monthly_folder, f"2018_monthly.csv")
monthly_df.to_csv(mon_out_path, index=False)

print(f"Saved 2018 Monthly Visitors csv file")
monthly_df

Saved 2018 Monthly Visitors csv file


,Country,January,February,March,April,May,June,July,August,September,October,November,December
0,South Korea,"198,145","156,555","122,387","114,973","113,488","111,640","127,190","129,734","125,236","122,390","127,935","174,578"
1,China,"111,344","146,536","114,549","109,789","78,071","85,800","119,990","106,789","102,374","87,243","97,513","97,964"
2,Usa,"109,154","84,835","90,961","85,573","92,754","94,567","91,684","65,567","59,600","76,090","79,089","104,597"
3,Japan,"57,038","60,262","63,878","49,021","45,744","40,119","50,591","65,136","53,342","45,108","47,776","53,806"
4,Australia,"30,924","19,480","23,623","25,292","21,279","19,663","20,817","17,578","20,551","21,167","22,460","36,994"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Peru,82,82,104,84,59,58,69,107,87,112,87,107
64,Luxembourg,84,81,98,95,72,51,90,97,34,47,63,118
65,Venezuela,37,18,36,33,24,25,35,26,34,61,50,44
66,Andorra,38,25,35,30,44,50,47,2,-,2,,-


In [9]:
# Store Special Categories Data

cat_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/special_category"

cat_df = combined_df.head(2)
cat_df['Country'] = cat_df['Country'].replace("T O T A L", "TOTAL")
cat_df['Country'] = cat_df['Country'].replace("Overseas Filipinos***", "Overseas Filipinos")
cat_df["Country"] = cat_df["Country"].astype(str).str.strip().str.title()
cat_df["Country"] = cat_df["Country"].apply(clean_country_name)

cat_df = cat_df.iloc[:, :-4]

cat_out_path = os.path.join(cat_folder, f"2018_category.csv")
cat_df.to_csv(cat_out_path, index=False)

print(f"Saved 2018 Category csv file")


Saved 2018 Category csv file


/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_64531/2616826766.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df['Country'] = cat_df['Country'].replace("T O T A L", "TOTAL")
/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_64531/2616826766.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df['Country'] = cat_df['Country'].replace("Overseas Filipinos***", "Overseas Filipinos")
/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_64531/2616826766.py:8: SettingWithCo

In [10]:
year_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/yearly_total"

year_df = combined_df[['Country', 'Total', 'Percentage', 'Previous Total', 'Growth Rate']]
year_df = year_df.iloc[2:]

year_df["Previous Total"] = (
    year_df["Previous Total"]
        .astype(str)
        .str.replace('"', '', regex=False)
        .str.replace(',', '', regex=False)   # REMOVE COMMAS
        .str.strip()
)

year_df["Previous Total"] = year_df["Previous Total"].astype(float)

year_df["Growth Rate"] = pd.to_numeric(year_df["Growth Rate"], errors = 'coerce')
year_df["Percentage"] = pd.to_numeric(year_df["Percentage"], errors = 'coerce')

year_path = os.path.join(year_folder, f"2018_year.csv")
year_df.to_csv(year_path, index=False)

print(f"Saved 2018 Year csv file")

Saved 2018 Year csv file
